In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro

import warnings
warnings.filterwarnings(action = 'ignore' )

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 100)

In [2]:
df = pd.read_excel("Consumer Data set - Baby.xlsx")

# <font color='blue'>Looking at head and tail to get quick snapshot of the data</font>

In [3]:
df.head()

,Need State,Category,Sub Category,Segment,Sub Segment,Manufacturer New,Actual_Month,$,Units
0,BABY CARE,BABY & KIDS TOILETRIES,ALL OTHERS,ALL OTHERS,ALL OTHERS,All Other,2016-11-01,890597.600,170520.000
1,BABY CARE,BABY & KIDS TOILETRIES,ALL OTHERS,ALL OTHERS,ALL OTHERS,All Other,2016-12-01,4933059.200,993499.200
2,BABY CARE,BABY & KIDS TOILETRIES,ALL OTHERS,ALL OTHERS,ALL OTHERS,All Other,2017-01-01,3648185.600,687198.400
3,BABY CARE,BABY & KIDS TOILETRIES,ALL OTHERS,ALL OTHERS,ALL OTHERS,All Other,2017-02-01,3629591.200,659757.600
4,BABY CARE,BABY & KIDS TOILETRIES,ALL OTHERS,ALL OTHERS,ALL OTHERS,All Other,2017-03-01,4751350.400,890494.400


In [4]:
df.tail()

,Need State,Category,Sub Category,Segment,Sub Segment,Manufacturer New,Actual_Month,$,Units
1986,BABY CARE,BABY WIPES,BABY WIPES,BABY WIPES,HAND & FACE WIPES,JPC,2019-06-01,1243797.600,333899.200
1987,BABY CARE,BABY WIPES,BABY WIPES,BABY WIPES,HAND & FACE WIPES,JPC,2019-07-01,924490.400,249528.000
1988,BABY CARE,BABY WIPES,BABY WIPES,BABY WIPES,HAND & FACE WIPES,JPC,2019-08-01,919956.800,250286.400
1989,BABY CARE,BABY WIPES,BABY WIPES,BABY WIPES,HAND & FACE WIPES,JPC,2019-09-01,1185224.800,333712.000
1990,BABY CARE,BABY WIPES,BABY WIPES,BABY WIPES,HAND & FACE WIPES,JPC,2019-10-01,801342.400,221572.000


#  <font color='blue'>Shape</font>

In [5]:
print("Total no. of rows: ", df.shape[0])
print("Total no. of Columns: ", df.shape[1])

Total no. of rows:  1991
Total no. of Columns:  9


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991 entries, 0 to 1990
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Need State        1991 non-null   object        
 1   Category          1991 non-null   object        
 2   Sub Category      1991 non-null   object        
 3   Segment           1991 non-null   object        
 4   Sub Segment       1991 non-null   object        
 5   Manufacturer New  1991 non-null   object        
 6   Actual_Month      1991 non-null   datetime64[ns]
 7   $                 1991 non-null   float64       
 8   Units             1991 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 140.1+ KB


**Observation 1**

**1.** Total 7 objects and 2 Int columns

**2.** no null values can be seen

In [7]:
# Renaming '$' to Total Sale.
print(f"Orignal Columns {df.columns}")
df.rename(columns={"$":"Total Sale"}, inplace=True)
print(f"Columns After Renaming {df.columns}")

Orignal Columns Index(['Need State', 'Category', 'Sub Category', 'Segment', 'Sub Segment',
       'Manufacturer New', 'Actual_Month', '$', 'Units'],
      dtype='object')
Columns After Renaming Index(['Need State', 'Category', 'Sub Category', 'Segment', 'Sub Segment',
       'Manufacturer New', 'Actual_Month', 'Total Sale', 'Units'],
      dtype='object')


In [8]:
df.describe(include=['object']).T

,count,unique,top,freq
Need State,1991,1,BABY CARE,1991
Category,1991,2,BABY & KIDS TOILETRIES,1664
Sub Category,1991,4,BABY TOILETRIES,1514
Segment,1991,10,BABY HAIR CARE,406
Sub Segment,1991,15,BABY GIFT SETS,210
Manufacturer New,1991,6,All Other,536


**Observation 3:**

**Category :**

* In Category **BABY & KIDS TOILETRIES** is the Highest freq.

**BABY TOILETRIES :**

* In Sub-category **BABY TOILETRIES** is the highest sold/demanded product.

**Segment :**

* In segment **BABY HAIR CARE** has the highest freq.

**Sub Segment :**

* In Sub Segment **BABY GIFT SETS** has the highest freq.

**Manufacturer New :**

* In Manufacturer New	**All Other** is the highest producer/manfacture.

**Actual_Month :**

* In Actual_Month **01-05-2019** is the highest no. of occurance Month



#  <font color='blue'>Data Cleaning</font>

### Step 1- Check Duplicate Values

In [9]:
df1 = df.copy()

In [10]:
dup = df1.duplicated().sum()
print('No. of duplicate rows :', dup)

No. of duplicate rows : 0


**Observation 4:**

* No Duplicate values

### Step 2 - Check Null Values

In [11]:
df1.isnull().sum()*100/len(df)

Need State         0.000
Category           0.000
Sub Category       0.000
Segment            0.000
Sub Segment        0.000
Manufacturer New   0.000
Actual_Month       0.000
Total Sale         0.000
Units              0.000
dtype: float64

**Observation 5:**
    
* No Null values    

In [12]:
# To get an idea how many unique categories we have to take our analysis further
df.select_dtypes(exclude=['float64']).nunique()

Need State           1
Category             2
Sub Category         4
Segment             10
Sub Segment         15
Manufacturer New     6
Actual_Month        36
dtype: int64

In [13]:
## from above results we can clearly see there is no variablity/ variance in Need State.
## The dataset looks Heirarichal in nature like we have 2 categories inside that we have 4 sub category.
## which is further divided into 10 segments and then into 15 sub segments.

#  <font color='blue'>Creating the derived columns from Actual_Month column to get more understanding on the sales based on Month, Quarter and Year.</font>

In [97]:
# Deriving Year from Actual_Month
df["Year"] = df["Actual_Month"].dt.year

# Deriving Month from Actual_Month
df["Month"] = df["Actual_Month"].dt.month

# Deriving Quarter from Actual_Month
df["Quarter"] = df["Actual_Month"].dt.quarter

df["Year_Month"] = df["Year"].astype(str) + "_" + df["Month"].astype(str)

# <font color='blue'>Comparision of sales over Year/ Month/ Quarter to analyse trend </font>

In [15]:
import plotly.express as px

In [101]:
## Created a user defined Function for Trend Analysis

def sale_trend_analysis(df,time_based_column="Actual_Month",column_based = "Category",sort_by = None):
    grouped_df = df.groupby([column_based,time_based_column]).sum().reset_index()
    if sort_by is not None:
        grouped_df = grouped_df.sort_values(by=sort_by).reset_index(drop=True)
    for val in grouped_df[column_based].unique():
        title = f'{time_based_column} wise sales trend for {column_based} {val}'
        y_label = f'Sales of {val} over Year'
        temp = grouped_df[grouped_df[column_based]== val][[time_based_column,'Total Sale']]
        fig = px.line(temp, time_based_column, "Total Sale",title = title)
        fig.show()
        

## Trend Analysis Category wise based on Years

In [34]:
sale_trend_analysis(df,time_based_column = 'Year',column_based = "Category")

**Obsevations :**

For Category wise we can clearly see their is **fall in sales from 2017** for both Baby & Kids Toiletris and Baby Wipes

In [17]:
df.columns

Index(['Need State', 'Category', 'Sub Category', 'Segment', 'Sub Segment',
       'Manufacturer New', 'Actual_Month', 'Total Sale', 'Units', 'Year',
       'Month', 'Quarter'],
      dtype='object')

## Trend Analysis Sub Category wise based on Years

In [35]:
sale_trend_analysis(df,time_based_column = 'Year',column_based = "Sub Category")

**Observation :**

In Sub Category we can see that their is fall in sales in all the products from 2017 except Baby Toiletries whose fall in sales happened in 2018 .

## Trend Analysis Segment wise based on Years

In [36]:
sale_trend_analysis(df,time_based_column = 'Year',column_based = "Segment")

**Observation :**

In Segment we can see that their is fall in sales in all the products from 2017 except Body care, Baby Cleansers and Baby Gift Sets whose fall in sales happened in 2018 .

## Trend Analysis Sub Segment wise based on Years

In [37]:
sale_trend_analysis(df,time_based_column = 'Year',column_based = "Sub Segment")

**Observation :**

Below are the observations according to Sub Segment wise:

**(1) ALL OTHERS :**  Fall in sales happened from 2017

**(2) BABY MILK/LOTIONS :** Fall in sales happened from 2018

**(3) BABY OIL :** Slight Fall in sales happened from 2017 but steeper fall in sales started from 2018

**(4) BABY CLEANSERS LIQUIDS :** Fall in sales happened from 2018

**(5) BABY COLOGNE :** Fall in sales happened from 2017

**(6) BABY GIFT SETS :** Fall in sales happened from 2018

**(7) BABY CONDITIONER :** Slight Fall in sales happened from 2017 but steeper fall in sales started from 2018

**(8) BABY CRADLE CAP :** Rise in sales happpened from 2018

**(9) BABY SHAMPOO :** Fall in sales happened from 2017

**(10) BABY STYLING :** Fall in sales happened from 2017

**(11) BABY POWDER :** Fall in sales happened from 2017

**(12) BABY NAPPY CREAMS :** Fall in sales happened from 2018

**(13) KIDS HAIR CARE :** Fall in sales happened from 2017

**(14) BOTTOM WIPES :** Fall in sales happened from 2017

**(15) HAND & FACE WIPES :** No fall in sales can be seen 




In [94]:
df["Sub Segment"].unique()

array(['ALL OTHERS', 'BABY MILK/LOTIONS', 'BABY OIL',
       'BABY CLEANSERS LIQUIDS', 'BABY COLOGNE', 'BABY GIFT SETS',
       'BABY CONDITIONER', 'BABY CRADLE CAP', 'BABY SHAMPOO',
       'BABY STYLING', 'BABY POWDER', 'BABY NAPPY CREAMS',
       'KIDS HAIR CARE', 'BOTTOM WIPES', 'HAND & FACE WIPES'],
      dtype=object)

## Trend Analysis Category wise based on Quarter

In [38]:
sale_trend_analysis(df,time_based_column = 'Quarter',column_based = "Category")

**Obsevations :**

For Category wise we can clearly see their is a steep fall in sales in 3rd Quarter

## Trend Analysis Sub Category wise based on Quarter¶

In [39]:
sale_trend_analysis(df,time_based_column = 'Quarter',column_based = "Sub Category")

**Observations :**

For Sub Category wise also we can clearly see their is a steep fall in sales in 3rd Quarter 


## Trend Analysis Segment wise based on Quarter¶

In [40]:
sale_trend_analysis(df,time_based_column = 'Quarter',column_based = "Segment")

**Observations :**

Below are the Observations for Segment wise :

**(1) ALL OTHERS :** Fall in sales in 3rd Quater

**(2) BABY BODY CARE :** Fall in sales from 1st Quarter

**(3) BABY CLEANSERS :** Fall in sales in 3rd Quarter

**(4) BABY COLOGNE :** Fall in sales in 3rd Quarter

**(5) BABY GIFT SETS :** Fall in sales from 1st Quater steeper fall can be seen from 3rd Quarter

**(6) BABY HAIR CARE :** Fall in sales from 2nd Quater steeper fall can be seen from 3rd Quarter

**(7) BABY POWDER :** Fall in sales in 3rd Quarter

**(8) NAPPY CREAMS :** Fall in sales from 1st Quarter steeper fall can be seen from 3rd Quarter

**(9) KIDS HAIR CARE :** Increase in sales from 2nd Quarter then steeper fall can be seen from 3rd Quarter

**(10) BABY WIPES :** Fall in sales from 1st Quarter steeper fall can be seen from 3rd Quarter

In [107]:
df["Segment"].unique()

array(['ALL OTHERS', 'BABY BODY CARE', 'BABY CLEANSERS', 'BABY COLOGNE',
       'BABY GIFT SETS', 'BABY HAIR CARE', 'BABY POWDER', 'NAPPY CREAMS',
       'KIDS HAIR CARE', 'BABY WIPES'], dtype=object)

## Trend Analysis Sub Segment wise based on Quarter

In [41]:
sale_trend_analysis(df,time_based_column = 'Quarter',column_based = "Sub Segment")

**Observation :**

Below are the observations according to Sub Segment wise:

**(1) ALL OTHERS :**  Fall in sales happened from 3rd Quarter

**(2) BABY MILK/LOTIONS :** Fall in sales happened from 1st Quarter then a small rise in 3rd Quarter.

**(3) BABY OIL :** Fall in sales happened from 2nd Quarter

**(4) BABY CLEANSERS LIQUIDS :** Fall in sales happened from 3rd Quarter

**(5) BABY COLOGNE :** Fall in sales happened from 3rd Quarter

**(6) BABY GIFT SETS :** Fall in sales from 1st Quarter steeper fall can be seen from 3rd Quarter

**(7) BABY CONDITIONER :** Fall in sales happened from 3rd Quarter

**(8) BABY CRADLE CAP :** Fall in sales happened from 2nd Quarter

**(9) BABY SHAMPOO :** Fall in sales from 2nd Quarter steeper fall can be seen from 3rd Quarter

**(10) BABY STYLING :** Fall in sales from 1st Quarter steeper fall can be seen from 3rd Quarter

**(11) BABY POWDER :** Fall in sales happened from 3rd Quarter

**(12) BABY NAPPY CREAMS :** Fall in sales happened from 3rd Quarter

**(13) KIDS HAIR CARE :** Fall in sales happened from 3rd Quarter

**(14) BOTTOM WIPES :** Fall in sales from 1st Quarter steeper fall can be seen from 3rd Quarter

**(15) HAND & FACE WIPES :** Fall in sales from 1st Quarter then rise can be seen from 2nd Quarter 

## Trend Analysis Category wise based on Months

In [43]:
sale_trend_analysis(df,time_based_column = 'Month',column_based = "Category")

**Observations**

* Even if their is an over all fall in sales over the year but we can see peak sales months after every 3 months sales cycle 

* Highest dip in sales can be seen in 11th month

## Trend Analysis Sub Category wise based on Months

In [44]:
sale_trend_analysis(df,time_based_column = 'Month',column_based = "Sub Category")

**Observations**

* Even if their is an over all fall in sales over the year but we can see peak sales months after every 3 months sales cycle 

* Highest dip in sales can be seen in 11th month for all the Sub Categories

## Trend Analysis Segment wise based on Months

In [45]:
sale_trend_analysis(df,time_based_column = 'Month',column_based = "Segment")

**Observations**

* Even if their is an over all fall in sales over the year but we can see peak sales months after every 3 months sales cycle 

* Highest dip in sales can be seen in 11th month for all the Segments

## Trend Analysis Sub Segment wise based on Months

In [46]:
sale_trend_analysis(df,time_based_column = 'Month',column_based = "Sub Segment")

**Observations**

* Even if their is an over all fall in sales over the year but we can see peak sales months after every 3 months sales cycle 

* Highest dip in sales can be seen in 11th month for all the Sub Segments

In [47]:
df.columns

Index(['Need State', 'Category', 'Sub Category', 'Segment', 'Sub Segment',
       'Manufacturer New', 'Actual_Month', 'Total Sale', 'Units', 'Year',
       'Month', 'Quarter'],
      dtype='object')

In [48]:
temp_grouped_df = df.groupby(["Category","Month","Year"]).sum()['Total Sale'].reset_index()

In [49]:
temp_grouped_df["Month_Year"] = temp_grouped_df["Month"].astype(str) + "_" + temp_grouped_df["Year"].astype(str)

In [51]:
df["Month_Year"] = df["Month"].astype(str) + "_" + df["Year"].astype(str)

In [63]:
df.sort_values(by=["Year","Month"])["Month_Year"].unique()

array(['11_2016', '12_2016', '1_2017', '2_2017', '3_2017', '4_2017',
       '5_2017', '6_2017', '7_2017', '8_2017', '9_2017', '10_2017',
       '11_2017', '12_2017', '1_2018', '2_2018', '3_2018', '4_2018',
       '5_2018', '6_2018', '7_2018', '8_2018', '9_2018', '10_2018',
       '11_2018', '12_2018', '1_2019', '2_2019', '3_2019', '4_2019',
       '5_2019', '6_2019', '7_2019', '8_2019', '9_2019', '10_2019'],
      dtype=object)

## Trend Analysis Category wise on Year_Month Basis

In [102]:
a = sale_trend_analysis(df,time_based_column = 'Year_Month',column_based = "Category",sort_by=["Year","Month"])

## Trend Analysis Sub Category wise on Year_Month Basis

In [104]:
sale_trend_analysis(df,time_based_column = 'Year_Month',column_based = "Sub Category",sort_by=["Year","Month"])

In [105]:
sale_trend_analysis(df,time_based_column = 'Year_Month',column_based = "Segment",sort_by=["Year","Month"])

In [106]:
sale_trend_analysis(df,time_based_column = 'Year_Month',column_based = "Sub Segment",sort_by=["Year","Month"])

In [50]:

for val in temp_grouped_df.Category.unique():
    # print(val)
    temp = temp_grouped_df.loc[temp_grouped_df['Category'] == val]
    title = f'Month wise sales trend for {val}'
    y_label = f'Cumulative Sales of {val} over month'
    # figures.append(px.line(temp, x="Month_Year", y="Total Sale"))
    # df.groupby(df.loc[df['Category']== val].Month).sum()['Total Sale'].plot(kind = 'line', title = title, ylabel = y_label)
    fig = px.line(temp, "Month_Year", "Total Sale",title = f"Each Month Sales Trend for {val}")#.set_title(title,pad=20)
    fig.show()

In [81]:
df.groupby(df.loc[df['Category']== 'BABY WIPES'].Month).sum()['Total Sale']

Month
1.000    352653046.400
2.000    364272666.400
3.000    470105094.400
4.000    354192732.800
5.000    355734179.200
6.000    437163482.400
7.000    347424225.600
8.000    358300138.400
9.000    440238500.800
10.000   336585599.200
11.000   252318960.000
12.000   424344542.400
Name: Total Sale, dtype: float64

# <font color='blue'>Sale comparison by category and Year</font>

In [82]:
sale_comaprison_year_on_year = df.groupby(by = ["Category","Year"])["Total Sale"].sum().reset_index()
sale_comaprison_year_on_year.sort_values(by=["Category","Total Sale"],ascending=False)

,Category,Year,Total Sale
5,BABY WIPES,2017,1592202131.200
6,BABY WIPES,2018,1480581912.000
7,BABY WIPES,2019,1246753308.800
4,BABY WIPES,2016,173795816.000
1,BABY & KIDS TOILETRIES,2017,1452827982.400
2,BABY & KIDS TOILETRIES,2018,1417379336.800
3,BABY & KIDS TOILETRIES,2019,1145322180.000
0,BABY & KIDS TOILETRIES,2016,156833770.400


* By looking at above table we can crearly conclude that sales were maximum in Year 2017 and least in Year 2016
* Sales of BABY WIPES are always greater then BABY & KIDS TOILERIES when compare year on year.
* Both category follows relative same pattern on sales figure. Eg. in 2017 Sales was highest for both categories and in 2016 Sale was minimum in both categories.

# <font color='blue'>Sale comparison by Sub Category and Year</font>

In [ ]:
sale_comaprison_year_on_year = df.groupby(by = ["Sub Category","Year"])["Total Sale"].sum().reset_index()
temp_df = sale_comaprison_year_on_year.sort_values(by=["Sub Category"],ascending=False)
temp_df

In [ ]:
import plotly.express as px
fig = px.bar(temp_df, x="Sub Category", y="Total Sale", color="Year",barmode='group', text_auto=True,height=600)
fig.show()

In [ ]:
## From above chart and Table we can make following conclusions:
# Baby Wipes is most sald product in each year and has highest contribution in maximum sale.
# Baby Toiletries is second most sald product in each year followed by KIDS TOTILETRIES and has second highest contribution in maximum sale.

In [ ]:
df["Segment"].unique()

In [ ]:
df["Sub Segment"].unique()

# <font color='blue'>Demand Variability</font>
How stable is your customers’ demand?

Average Sales: µ
Standard Deviation:
Coefficient of Variation: CV = σ/µ
For SKUs with a high value of CV, you may face unstable customer demand that would lead to workload peaks, forecasting complexity and stock-outs.

In [91]:
# PATH
PATH = ''

In [92]:
# Columns
COLS_ITM = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
COLS_DATE = ['d_' + str(i) for i in range(1, 366)]

In [93]:
# Import
df = pd.read_csv(PATH + 'sales_train_evaluation.csv')
print("{:,} records for train data set".format(len(df)))
# Change id to match with price dataframe
df['id'] = df['store_id'] + '-' + df['item_id']
# Set index
df.set_index(COLS_ITM, inplace = True)
# Scope = Year 1
df = df[COLS_DATE]

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'sales_train_evaluation.csv'

In [ ]:
# Import Calendar
df_calendar = pd.read_csv(PATH + 'calendar.csv')
# Date to Week
dict_week = dict(zip(df_calendar.d.values, df_calendar.wm_yr_wk.values))
df_calendar.head()

In [ ]:
# Import
df_price = pd.read_csv(PATH + 'sell_prices.csv')
print("{:,} records for sales price".format(len(df_price)))
# SKU Index
df_price['item_store_id'] = df_price['store_id'] + '-' + df_price['item_id']
# Pricing
df_price = df_price.pivot(index='item_store_id', columns='wm_yr_wk', values='sell_price').fillna(0)
# Matrix from pivot 
matrix_price = df_price.to_numpy()
# Dict Matrix Index
pr_n = dict(zip(df_price.index, range(len(df_price.index))))
pr_p = dict(zip(df_price.columns, range(len(df_price.columns))))
print("{:,} records for sales price pivot".format(len(df_price)))
df_price.head()

In [ ]:
# Mean
df['mean'] = df[COLS_DATE].mean(axis = 1)
# Standard
df['std'] = df[COLS_DATE].std(axis = 1)

# Remove items not sold during the first year
print("{:,} records for the full scope".format(len(df)))
df = df[df['mean']>0]
print("{:,} records for after filter".format(len(df)))
df.reset_index(inplace = True)
# Sigma
df.head()

In [ ]:
 # Total Units
df['units'] = df[COLS_DATE].sum(axis = 1)
# Turnover = Units x Price
df['TO'] = 0 
for col in COLS_DATE:
    df['TO'] = df['TO'] + df[col] * df[['id', col]].apply(
        lambda t: matrix_price[pr_n[t['id']], pr_p[dict_week[col]]], axis = 1)
# Sigma
df.head()

In [ ]:
# Coefficient of Variation
df['CV'] = df['std']/df['mean']
df.head()

In [ ]:
# ABC SKU-LEVEL
df_abc = df[df['cat_id']=='HOBBIES'].drop(['mean', 'std', 'CV'], axis = 1).copy()

In [ ]:
df_abc

In [ ]:
df_abc = df_abc.groupby(['item_id', 'dept_id', 'cat_id']).sum()

In [ ]:
df_abc

In [ ]:
COLS_DATE

In [ ]:
df_abc['mean'] = df_abc[COLS_DATE].mean(axis = 1)

In [ ]:
df_abc

In [ ]:
# ABC SKU-LEVEL
df_abc = df[df['cat_id']=='HOBBIES'].drop(['mean', 'std', 'CV'], axis = 1).copy()
df_abc = df_abc.groupby(['item_id', 'dept_id', 'cat_id']).sum()

# Calculate mean and standard deviation
# Mean
df_abc['mean'] = df_abc[COLS_DATE].mean(axis = 1)
# Filter out the reference withou sales
df_abc = df_abc[df_abc['mean']>0]
# Standard
df_abc['std'] = df_abc[COLS_DATE].std(axis = 1)
# Coefficient of Variation
df_abc['CV'] = df_abc['std']/df_abc['mean']
df_abc.reset_index(inplace = True)
# Normalility Test
df_abc['Normality_p'] = df_abc[COLS_DATE].apply(lambda row : stats.shapiro(row)[1], axis = 1)
alpha = 0.001
df_abc['Not_Normal'] = df_abc['Normality_p'] < alpha

# # ABC SKU-LEVEL
df_abc = df_abc.drop(COLS_DATE, axis =1).copy()
df_abc['TO%'] = (100*df_abc['TO']/df_abc['TO'].sum())
# Sort 
df_abc.sort_values(['TO%'], ascending = False, inplace = True, ignore_index=True)
df_abc['TO%_CS'] = df_abc['TO%'].cumsum()

# A, B, C on SKU Number
n_sku = len(df_abc)
n_a, n_b = int(0.05*n_sku), int(0.5*n_sku)
df_abc['SKU_ID'] = pd.Series(range(1, len(df_abc))).astype(int)
df_abc['SKU_%'] = (100 * pd.Series(range(1, len(df_abc))) / len(df_abc))
df_abc['ABC'] = pd.Series(range(len(df_abc))).apply(lambda t: 'A' if t <= n_a-1 else 'B' if t <= n_b-1 else 'C')
# A, B, C on turnover
to_a, to_b = df_abc[df_abc['SKU_ID']==n_a]['TO%'].max(), df_abc[df_abc['SKU_ID']==n_b]['TO%'].max()

print("{:,} unique SKU".format(len(df_abc)))
df_abc.head()

In [ ]:
df

In [ ]:
## Grouping retail sku's to identify datewise sales

retail_grouped= df.groupby(['Description','date']).agg(total_sale=('Quantity','sum')).reset_index()

In [ ]:
df.columns

In [ ]:
df_abc = df.groupby(['Need State', 'Category', 'Sub Category', 'Segment', 'Sub Segment','Actual_Month']).sum()

In [ ]:
df_abc

In [ ]:
df_abc.index

In [ ]:
df_abc['mean'] = df_abc.iloc["Actual_Month"].mean(axis = 1)

In [ ]:
# ABC SKU-LEVEL
df_abc = df[df['cat_id']=='HOBBIES'].copy()
df_abc = df_abc.groupby(['item_id', 'dept_id', 'cat_id']).sum()

# Calculate mean and standard deviation
# Mean
df_abc['mean'] = df_abc[COLS_DATE].mean(axis = 1)
# Filter out the reference without sales
df_abc = df_abc[df_abc['mean']>0]
# Standard Deviation
df_abc['std'] = df_abc[COLS_DATE].std(axis = 1)
# Coefficient of Variation
df_abc['CV'] = df_abc['std']/df_abc['mean']
df_abc.reset_index(inplace = True)
# # ABC SKU-LEVEL
df_abc = df_abc.drop(COLS_DATE, axis =1).copy()
df_abc['TO%'] = (100*df_abc['TO']/df_abc['TO'].sum())
# Sort 
df_abc.sort_values(['TO%'], ascending = False, inplace = True, ignore_index=True)
df_abc['TO%_CS'] = df_abc['TO%'].cumsum()

# A, B, C on SKU Number
n_sku = len(df_abc)
n_a, n_b = int(0.05*n_sku), int(0.5*n_sku)
df_abc['SKU_ID'] = pd.Series(range(1, len(df_abc))).astype(int)
df_abc['SKU_%'] = (100 * pd.Series(range(1, len(df_abc))) / len(df_abc))
df_abc['ABC'] = pd.Series(range(len(df_abc))).apply(lambda t: 'A' if t <= n_a-1 else 'B' if t <= n_b-1 else 'C')
# A, B, C on turnover
to_a, to_b = df_abc[df_abc['SKU_ID']==n_a]['TO%'].max(), df_abc[df_abc['SKU_ID']==n_b]['TO%'].max()